<a href="https://colab.research.google.com/github/GerryChen117/Python-Coding-Challenges/blob/main/MOST_UPDATED_Stock_Statement_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Statement Generator

For ease of coding and testing, I used Google Colab to write this document. Link to the Colab: https://colab.research.google.com/drive/1Eve6tqEf2HjH1q9MYKyPSSRX3WWpgxx6?usp=sharing

In [1]:
# Mount the Drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


For all my process work, see: https://colab.research.google.com/drive/1KlF8hW0gAE8GL0pJPROoKITpUWzoLp0i?usp=sharing 

In [3]:
# Import the libraries I will use. 
import json
import time

All that is required for the user of this code to do is change the 'file_input_directory' (containing the json file) and the 'file_output_directory' (where they want to store the text file). 

In [4]:
# Load the data into a variable from a json file. 
file_input_directory = "/content/drive/MyDrive/Forma/input.json"
f = open(file_input_directory, "r")
data = json.load(f)

# This variable is important to tell the algorithm where to spit out the text file. 
file_output_directory = "/content/drive/MyDrive/Forma/output.txt"

### The relevant 'universal' Python code begins here. 

In [6]:
# Import the libraries I will use. 
import json
import time

class stock_info_class:
  """ A class used to hold the information of each stock. Stocks will be easily referenced by 'ticker' or 'stock' name via a dictionary. """ 

  def __init__(self, ticker, price, shares):
    self.stock_name = ticker
    self.price_per_share = float(price)
    self.num_shares = int(shares)
  
  def __call__(self):
    return "   - %d shares of %s at $%.2f per share\n" %(self.num_shares, self.stock_name, self.price_per_share)

class quote_class:
  """ A class that holds the items to be inserted into the text file for every date. 
      A dictionary will be used to store these classes, referenced by date, and will be iterated through to generate the text file. 
      Many of these quotes are generated right when their respective actions are performed. """

  def __init__(self, just_the_date, transaction_quote, dividend_quote, stock_quotes):
    self.transaction_quote = [transaction_quote]  # Initialized as an array to allow for multiple transaction statements to fall under the same date even if they are of different actions.
    self.dividend_quote = dividend_quote
    self.stock_quotes = stock_quotes
    self.just_the_date = just_the_date

# The main stock statement generation class. 
class StockStateGen:
  #def __init__(self, file_input_directory, file_output_directory):
  def __init__(self):
    """ This class would have accepted the parameters 'data' as data loaded from a json file and 'output_directory' as a string. 
        In my most recent version of this, the user is asked for inputs about the input and output paths and the class does the rest. """

    # Load in the data, containing both the user's 'actions' and the 'stock_actions', into a global class variable.
    file_input_directory = input("Where is your file input directory (with json file): ")  
    f = open(file_input_directory, "r")
    self.data = json.load(f)

    # Initiate a Nested Dictionary containing 'stock_info_class' for each stock (stock information like name, price per share, and number of shares).
    self.stock_info = {}

    # Initiate an accumulator to hold overall dividend income. 
    self.total_dividend = 0.00

    # Initiate a Nested Dictionary that holds 'quote_class' objects, which contains all the statements we wish to print for each date.
    self.dict_of_actions = {}

    # Load a global class variable with the output location for the text file.
    file_output_directory = input("Where is your file output directory (for text file): ")   
    self.output_directory = file_output_directory

  # Checks if stock is in the nested dictionary, "stock_info", already. 
  def stock_exist(self, ticker):
    if ticker in self.stock_info:
      return True
    else:
      return False

  # If stock is not in nested dictionary of "stock_info", initialize a stock class for it in the dictionary. Only called when the user first buys a new stock. 
  def new_stock_info(self, ticker, price, shares):
    self.stock_info[ticker] = stock_info_class(ticker, price, shares)
    return None

  # Buying stocks could change the price per share of the stock in your inventory. This function accounts for that. 
  # It calculates total amount of money you put into this stock and divides it by the total number of shares you have now. 
  def price_per_share_adjustments(self, ticker, price, shares):
    past_shares = self.stock_info[ticker].num_shares
    past_price = self.stock_info[ticker].price_per_share
    
    total_shares = past_shares + int(shares)
    total_investment = (past_shares*past_price) + float(price)*int(shares)
    return total_investment/total_shares

  # Increase number of shares in specified stock. Might add feature to know total money spent later. Generates the respective buy stock quote. 
  def buy_stock(self, ticker, price, shares):
    # If the stock exists in the "stock_info" dictionary, we simply adjust price per share if applicable and increase its number of shares.
    if self.stock_exist(ticker):
      # change stock price if necessary
      self.stock_info[ticker].price_per_share = self.price_per_share_adjustments(ticker, price, shares)
      self.stock_info[ticker].num_shares += int(shares)
    
    # If the stock is not in the "stock_info" dictionary, we initiate the class using the information of purchase. 
    else:
      self.new_stock_info(ticker, price, shares)
    return "   - You bought %s shares of %s at a price of $%.2f per share\n" %(shares, ticker, float(price))

  # Decrease the number of shares in specified stock. Generates the respective sell stock quote. 
  def sell_stock(self, ticker, price, shares):
    self.stock_info[ticker].num_shares -= int(shares)
    # To account for profit or loss when selling stocks. 
    past_shares = self.stock_info[ticker].price_per_share
    profit = float(price)*float(shares) - float(shares)*past_shares

    # Remove stock_info entry if no shares remaining, assuming you can't sell more than you bought. 
    if self.stock_info[ticker].num_shares == 0:
      self.stock_info.pop(ticker, None)

    # To account for different scenarios:
    if profit > 0:
      return "   - You sold %s shares of %s at a price of $%.2f per share for a profit of $%.2f\n" %(shares, ticker, float(price), float(profit))
    elif profit < 0:
      return "   - You sold %s shares of %s at a price of $%.2f per share for a loss of $%.2f\n" %(shares, ticker, float(price), float(profit))
    else:
      return "   - You sold %s shares of %s at a price of $%.2f per share with neither a profit or loss of $%.2f\n" %(shares, ticker, float(price), float(profit))

  # Convert the dates (strings) into a format available for comparison.
  def date_format(self, date):
    try:
      python_date = time.strptime(date, "%Y/%m/%d %H:%M:%S")
    except:
      python_date = time.strptime(date, "%Y/%m/%d")
    return python_date

  # Changes the price per share of the specified stock. Used when splits occur. 
  def change_ppshare(self, ticker, new_price):
    self.stock_info[ticker].price_per_share = float(new_price)
    return None

  # Changes the number of shares of the specified stock. Used when splits occur. 
  def change_numshare(self, ticker, new_num):
    self.stock_info[ticker].num_shares = int(new_num)
    return None

  # Used to split the specified stock based on the split ratio when provided. 
  def stock_split(self, ticker, split_ratio):
    try:
      price = self.stock_info[ticker].price_per_share
      quantity = self.stock_info[ticker].num_shares

      new_ppshare = price/float(split_ratio)
      new_numshare = quantity*int(split_ratio)

      self.change_ppshare(ticker, new_ppshare)
      self.change_numshare(ticker, new_numshare)
      return "   - %s split %s to 1, and you have %d shares\n" %(ticker, split_ratio, new_numshare)

    # To account for an exception when the user didn't buy any of the stocks that were split and the stock doesn't exist in the dictionary stock_info:
    except:
      return ""

  # Calculates the dividend for the specified stock when a 'stock_actions' requests it.
  def pay_dividend(self, ticker, dividend):
    try:
      quantity = self.stock_info[ticker].num_shares
      pay = float(dividend)*quantity
      self.total_dividend += pay
      return "   - %s paid out $%.2f dividend per share, and you have %d shares\n" %(ticker, float(dividend), quantity)

    # To account for an exception when the user didn't buy any of the stocks paying dividends and the stock doesn't exist in the dictionary stock_info:
    except:
      return ""

  # Displays the overall dividend income in the specified format.
  def show_total_dividend(self):
    return "   - $%.2f of dividend income\n" %(self.total_dividend)

  # This is the main function to process both user and stock actions. Respective quotes to add to the text file are returned.
  def transactions(self, act_iter):
    """ Since 'actions' and 'stock_actions' from the json file store different information, a KeyError may arise.
        The try and except statements help to differentiate between the two form of actions and allow for quotes to be processed respectively. """
    try:
      date = act_iter['date']
      dividend = act_iter['dividend']
      split = act_iter['split']
      ticker = act_iter['stock']

      # Assuming the stock actions are either a split or a dividend, not both at the same action iteration. 
      if split != "" and dividend == "":
        split_ratio = int(split)
        result = self.stock_split(ticker, split_ratio)

      if dividend != "" and split == "":
        dividendd = float(dividend)
        result = self.pay_dividend(ticker,dividendd)
      return result

    except:
      date = act_iter['date']
      action = act_iter['action']
      price = act_iter['price']
      ticker = act_iter['ticker']
      shares = act_iter['shares']

      # If stock exists, we perform buy or sell 
      if action == 'BUY':
        result = self.buy_stock(ticker, price, shares)
      else:
        result = self.sell_stock(ticker, price, shares)
      return result

  # Convert the dates (strings) into a format available for comparison.
  def timeformat(self, act_iter):
    return self.date_format(act_iter['date'])
  
  # Combine both 'actions' and 'stock_actions' from the json file into one array of dictionaries, where each dictionary is an action. 
  # Actions are sorted chronological order by their date. 
  def sort_actions_by_time(self, data):
    data_together = data['actions'] + data['stock_actions']
    sorted_data = sorted(data_together, key=self.timeformat) 
    return sorted_data

  # Initiates a quote_class for an action if this is the first time action performed at that date. 
  def generate_quote_class(self, just_the_date, transaction_quote, dividend_quote,stock_quotes):
    self.dict_of_actions[just_the_date] = quote_class(just_the_date, transaction_quote, dividend_quote, stock_quotes)
    return None

  # If a quote_class has already been initiated for a date, we simply update the quotes printed for that date to account for all actions performed.
  # Note that special consideration has been granted to the transaction_quote section, as all transactions in a day are noted down together. 
  def alter_quote_class(self, just_the_date, transaction_quote, dividend_quote, stock_quotes):
    self.dict_of_actions[just_the_date].stock_quotes = stock_quotes
    self.dict_of_actions[just_the_date].dividend_quote = dividend_quote
    self.dict_of_actions[just_the_date].transaction_quote.append(transaction_quote)
    return None

  # Build the Nested Dictionary (dict_of_actions).
  def build_dict_of_actions(self):
    """
    self.actions = data['actions']
    self.stock_actions = data['stock_actions']
    self.dict_of_actions = {}
    """
    # Method 1: We combine the two lists into one and sort by date. Then we do the things incrementally. 
    data = self.data
    sorted_data = self.sort_actions_by_time(data)
    prev_tuple_time = None
    # Example of act: {'date': '1992/07/14 11:12:30', 'action': 'BUY', 'price': '12.3', 'ticker': 'AAPL', 'shares': '500'}
    # The following is used to generate a quote_class per action: 
    for act in sorted_data:
      date_struct_time = self.timeformat(act)
      just_the_date = time.strftime("%Y-%m-%d", date_struct_time)
      # if the dates are the same, everything else is replaced but transaction_quote, which shows all the transactions at the same date. 
      #if self.dict_of_actions.has_key(just_the_date):
      if just_the_date in self.dict_of_actions:

        transaction_quote = self.transactions(act)
        dividend_quote = self.show_total_dividend()
        stock_quotes = []
        for stock_class in self.stock_info:
          stock_quotes.append(self.stock_info.get(stock_class)())  ## iterate through a dictionary and use call on each to display text, save to an array 
        self.alter_quote_class(just_the_date, transaction_quote, dividend_quote, stock_quotes)
      else:
        transaction_quote = self.transactions(act)
        dividend_quote = self.show_total_dividend()
        stock_quotes = []
        for stock_class in self.stock_info:
          stock_quotes.append(self.stock_info.get(stock_class)())
        # Generate the quote class
        self.generate_quote_class(just_the_date, transaction_quote, dividend_quote, stock_quotes)

    return None

  # Write to Text File by iterating through every Nested Dictionary (dict_of_actions) entry and fetching the respective quotes. 
  def write_to_text(self):
    with open(self.output_directory, 'w+') as reader:
      for each_quote_class in self.dict_of_actions:
        # To remove actions that don't effect transaction statements, ex. stock paying dividend when user doesn't own stock
        if self.dict_of_actions[each_quote_class].transaction_quote != [""]:
          date = self.dict_of_actions[each_quote_class].just_the_date
          transaction_quote = self.dict_of_actions[each_quote_class].transaction_quote
          dividend_quote = self.dict_of_actions[each_quote_class].dividend_quote
          stock_quotes = self.dict_of_actions[each_quote_class].stock_quotes

          reader.write("On %s, you have:\n" %(date))
          for itemz in stock_quotes: 
            reader.write(itemz)
          reader.write(dividend_quote)
          reader.write("  Transactions:\n")
          for itemz in transaction_quote:
            reader.write(itemz)
        else:
          continue
    return None
  
  # An easy way for users to implement everything above with one line of code. 
  def __call__(self):
    self.build_dict_of_actions()
    self.write_to_text()

In [7]:
#test1 = StockStateGen(data, file_output_directory)
test1 = StockStateGen()
test1()

Where is your file input directory (with json file): /content/drive/MyDrive/Forma/input.json
Where is your file output directory (for text file): /content/drive/MyDrive/Forma/output1.txt
